## Imports

In [519]:
import pandas as pd
import numpy as np
import hvplot.pandas
import yfinance as yf
import datetime


## Data Collection

In [520]:
# Define the stock symbol and create a Yahoo Finance ticker object
symbol = "CORN"
ticker = yf.Ticker(symbol)

# Get historical stock data for the last 5 years
historical_data = ticker.history(period="5y")

# Print the first few rows of the historical data
print(historical_data.head())

                                Open       High    Low  Close  Volume  \
Date                                                                    
2018-09-12 00:00:00-04:00  16.190001  16.200001  15.68  15.78  325800   
2018-09-13 00:00:00-04:00  15.760000  15.800000  15.63  15.70   55800   
2018-09-14 00:00:00-04:00  15.650000  15.730000  15.65  15.71   31600   
2018-09-17 00:00:00-04:00  15.680000  15.680000  15.58  15.59   55600   
2018-09-18 00:00:00-04:00  15.560000  15.560000  15.39  15.40   98700   

                           Dividends  Stock Splits  Capital Gains  
Date                                                               
2018-09-12 00:00:00-04:00        0.0           0.0            0.0  
2018-09-13 00:00:00-04:00        0.0           0.0            0.0  
2018-09-14 00:00:00-04:00        0.0           0.0            0.0  
2018-09-17 00:00:00-04:00        0.0           0.0            0.0  
2018-09-18 00:00:00-04:00        0.0           0.0            0.0  


## Daily Returns

In [521]:
# Calculate daily returns
historical_data['Daily_Return'] = historical_data['Close'].pct_change()

# Print the first few rows of the data with daily returns
print(historical_data.head())

                                Open       High    Low  Close  Volume  \
Date                                                                    
2018-09-12 00:00:00-04:00  16.190001  16.200001  15.68  15.78  325800   
2018-09-13 00:00:00-04:00  15.760000  15.800000  15.63  15.70   55800   
2018-09-14 00:00:00-04:00  15.650000  15.730000  15.65  15.71   31600   
2018-09-17 00:00:00-04:00  15.680000  15.680000  15.58  15.59   55600   
2018-09-18 00:00:00-04:00  15.560000  15.560000  15.39  15.40   98700   

                           Dividends  Stock Splits  Capital Gains  \
Date                                                                
2018-09-12 00:00:00-04:00        0.0           0.0            0.0   
2018-09-13 00:00:00-04:00        0.0           0.0            0.0   
2018-09-14 00:00:00-04:00        0.0           0.0            0.0   
2018-09-17 00:00:00-04:00        0.0           0.0            0.0   
2018-09-18 00:00:00-04:00        0.0           0.0            0.0   

    

In [522]:
# Convert to DataFrame and Reset Index
df = pd.DataFrame(historical_data)
df = df.reset_index()
df = df.drop(columns = ["Open", "High", "Low", "Volume", "Dividends", "Stock Splits", "Capital Gains"])
df.head()

,Date,Close,Daily_Return
0,2018-09-12 00:00:00-04:00,15.78,NaN
1,2018-09-13 00:00:00-04:00,15.70,-0.005070
2,2018-09-14 00:00:00-04:00,15.71,0.000637
3,2018-09-17 00:00:00-04:00,15.59,-0.007638
4,2018-09-18 00:00:00-04:00,15.40,-0.012187


## Annualized Returns

In [523]:

# Calculate the total return over the period
total_return = (1 + df['Daily_Return']).prod() - 1

# Determine the number of years in the investment horizon
years = (df['Date'].max() - df['Date'].min()).days / 365

# Calculate annualized return using the formula: ((1 + Total Return)^(1 / Years) - 1)
annualized_return = (pow(1 + total_return, 1 / years) - 1) * 100 

print(years)
print(annualized_return)

5.0
7.190512162825113


## Cumulative Returns

In [524]:
# Calculate Cumulative Return and add as a Column
df['Cumulative_Return'] = (1 + df['Daily_Return']).cumprod() - 1

df.head()

,Date,Close,Daily_Return,Cumulative_Return
0,2018-09-12 00:00:00-04:00,15.78,NaN,NaN
1,2018-09-13 00:00:00-04:00,15.70,-0.005070,-0.005070
2,2018-09-14 00:00:00-04:00,15.71,0.000637,-0.004436
3,2018-09-17 00:00:00-04:00,15.59,-0.007638,-0.012041
4,2018-09-18 00:00:00-04:00,15.40,-0.012187,-0.024081


## Annual Volatility 

In [525]:
# Calculate the standard deviation of daily returns
daily_volatility = df['Daily_Return'].std()

# Annualize the volatility
annual_volatility = daily_volatility * np.sqrt(252)

print(daily_volatility)
print(annual_volatility)

0.014002668772951124
0.22228547558663184


## Sharpe Ratio

In [526]:
# Calculate the average daily return
average_daily_return = df['Daily_Return'].mean()

# Assume a daily risk free rate
daily_risk_free_rate = 0.001

# Calculate the Sharpe ratio
sharpe_ratio = (average_daily_return - daily_risk_free_rate) / daily_volatility

print(average_daily_return)
print(sharpe_ratio)

0.0003743550672460483
-0.044680406492404255


## Sortino Ratio

In [527]:
# Calculate downside risk (standard deviation of negative returns)
negative_returns = df[df['Daily_Return'] < 0]['Daily_Return']
downside_risk = negative_returns.std()

# Calculate the Sortino ratio
sortino_ratio = (average_daily_return - daily_risk_free_rate) / downside_risk

print(sortino_ratio)

-0.06559586892568836


# Trading Algorithm

In [528]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

## Create Short and Long Window 

In [529]:
df = df.set_index(["Date"])

df['SMA_Short'] = df['Close'].rolling(window=69).mean()
df['SMA_Long'] = df['Close'].rolling(window=187).mean()
df.head()

,Close,Daily_Return,Cumulative_Return,SMA_Short,SMA_Long
Date,,,,,
2018-09-12 00:00:00-04:00,15.78,NaN,NaN,NaN,NaN
2018-09-13 00:00:00-04:00,15.70,-0.005070,-0.005070,NaN,NaN
2018-09-14 00:00:00-04:00,15.71,0.000637,-0.004436,NaN,NaN
2018-09-17 00:00:00-04:00,15.59,-0.007638,-0.012041,NaN,NaN
2018-09-18 00:00:00-04:00,15.40,-0.012187,-0.024081,NaN,NaN


In [530]:
# Create a binary target variable: 1 for buy signal, -1 for sell signal

df['Signal'] = np.where(df['SMA_Short'] > df['SMA_Long'], 1, -1)
df.head()

,Close,Daily_Return,Cumulative_Return,SMA_Short,SMA_Long,Signal
Date,,,,,,
2018-09-12 00:00:00-04:00,15.78,NaN,NaN,NaN,NaN,-1
2018-09-13 00:00:00-04:00,15.70,-0.005070,-0.005070,NaN,NaN,-1
2018-09-14 00:00:00-04:00,15.71,0.000637,-0.004436,NaN,NaN,-1
2018-09-17 00:00:00-04:00,15.59,-0.007638,-0.012041,NaN,NaN,-1
2018-09-18 00:00:00-04:00,15.40,-0.012187,-0.024081,NaN,NaN,-1


In [531]:
# Drop NaN values and shift target variable by one day
df['Signal'] = df['Signal'].shift(-1)
df = df.dropna()
df.head()

,Close,Daily_Return,Cumulative_Return,SMA_Short,SMA_Long,Signal
Date,,,,,,
2019-06-11 00:00:00-04:00,16.750000,0.017001,0.061470,15.619275,15.968128,-1.0
2019-06-12 00:00:00-04:00,16.799999,0.002985,0.064639,15.632464,15.973583,-1.0
2019-06-13 00:00:00-04:00,17.059999,0.015476,0.081115,15.651304,15.980856,-1.0
2019-06-14 00:00:00-04:00,17.250000,0.011137,0.093156,15.676812,15.989091,-1.0
2019-06-17 00:00:00-04:00,17.410000,0.009275,0.103295,15.704927,15.998823,-1.0


In [532]:
# Prepare data for machine learning
X = df.drop(columns = (["Close", "Daily_Return", "Cumulative_Return", "Signal"]))
y = df["Signal"]
display(X)
display(y)


,SMA_Short,SMA_Long
Date,,
2019-06-11 00:00:00-04:00,15.619275,15.968128
2019-06-12 00:00:00-04:00,15.632464,15.973583
2019-06-13 00:00:00-04:00,15.651304,15.980856
2019-06-14 00:00:00-04:00,15.676812,15.989091
2019-06-17 00:00:00-04:00,15.704927,15.998823
...,...,...
2023-09-01 00:00:00-04:00,23.342464,24.552353
2023-09-05 00:00:00-04:00,23.335073,24.534813
2023-09-06 00:00:00-04:00,23.315942,24.517433


Date
2019-06-11 00:00:00-04:00   -1.0
2019-06-12 00:00:00-04:00   -1.0
2019-06-13 00:00:00-04:00   -1.0
2019-06-14 00:00:00-04:00   -1.0
2019-06-17 00:00:00-04:00   -1.0
                            ... 
2023-09-01 00:00:00-04:00   -1.0
2023-09-05 00:00:00-04:00   -1.0
2023-09-06 00:00:00-04:00   -1.0
2023-09-07 00:00:00-04:00   -1.0
2023-09-08 00:00:00-04:00   -1.0
Name: Signal, Length: 1070, dtype: float64

In [533]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [534]:
# Scale the data and transform
scaler = StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [537]:
# Build and train a machine learning model
model = LogisticRegression(random_state = 1)
model.fit(X_train_scaled, y_train)

LogisticRegression(random_state=1)

In [538]:
# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
testing_report = classification_report(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print(testing_report)

Accuracy: 0.9205607476635514
              precision    recall  f1-score   support

        -1.0       1.00      0.85      0.92       114
         1.0       0.85      1.00      0.92       100

    accuracy                           0.92       214
   macro avg       0.93      0.93      0.92       214
weighted avg       0.93      0.92      0.92       214



C:\Users\glzvi\anaconda3\envs\dev\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
